In [ ]:
# imports
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from sklearn.metrics.pairwise import cosine_similarity
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import explained_variance_score, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [ ]:
import nltk
nltk.download('all')

In [ ]:
# # url extraction
# urls = ['https://www.carwow.co.uk/suzuki', 'https://www.carwow.co.uk/toyota','https://www.carwow.co.uk/honda', 'https://www.carwow.co.uk/hyundai', 
#         'https://www.carwow.co.uk/mazda', 'https://www.carwow.co.uk/mitsubishi', 'https://www.carwow.co.uk/nissan', 'https://www.carwow.co.uk/lexus', 
#         'https://www.carwow.co.uk/abarth', 'https://www.carwow.co.uk/alfa-romeo', 'https://www.carwow.co.uk/alpine', 'https://www.carwow.co.uk/audi', 
#         'https://www.carwow.co.uk/bmw', 'https://www.carwow.co.uk/byd', 'https://www.carwow.co.uk/citroen', 'https://www.carwow.co.uk/cupra',
#         'https://www.carwow.co.uk/dacia', 'https://www.carwow.co.uk/ds', 'https://www.carwow.co.uk/fiat', 'https://www.carwow.co.uk/ford', 
#         'https://www.carwow.co.uk/ineos', 'https://www.carwow.co.uk/jaguar', 'https://www.carwow.co.uk/jeep', 'https://www.carwow.co.uk/kgm-motors', 
#         'https://www.carwow.co.uk/kia', 'https://www.carwow.co.uk/land-rover', 'https://www.carwow.co.uk/lotus', 'https://www.carwow.co.uk/mazda',
#         'https://www.carwow.co.uk/mercedes', 'https://www.carwow.co.uk/mg', 'https://www.carwow.co.uk/mini', 'https://www.carwow.co.uk/ora',
#         'https://www.carwow.co.uk/peugeot', 'https://www.carwow.co.uk/polestar', 'https://www.carwow.co.uk/renault', 'https://www.carwow.co.uk/seat', 
#         'https://www.carwow.co.uk/skoda', 'https://www.carwow.co.uk/smart', 'https://www.carwow.co.uk/subaru', 'https://www.carwow.co.uk/tesla',
#         'https://www.carwow.co.uk/vauxhall', 'https://www.carwow.co.uk/Volkswagen','https://www.carwow.co.uk/volvo']
# def correction(data):
#   newdata=''
#   count=0
#   for i in data:
#     if count==1:
#        if i==' ':
#           newdata+='-'
#        else:
#           newdata+=i
#     elif i==' ':
#        count=1;
#   return newdata
# with open('6.csv', 'w', newline='', encoding='utf-8') as csvfile:
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#     writer.writeheader()
#     for url in urls:
#         response = requests.get(url)
#         soup = BeautifulSoup(response.text, 'html.parser')

#         cars = soup.find_all('div', {'class': 'card-generic__title'})

#         for car in cars:
#             car_name = car.text.strip()
#             model = correction(car_name)
#             newurl = f"{url}/{model}"

#             r1 = requests.get(newurl)
#             soup1 = BeautifulSoup(r1.text, 'html.parser')

#             # Extract engine data
#             engine_element = soup1.find('div', {'class': 'summary-list__item'})
#             engine = engine_element.find('dd').text.strip() if engine_element else 'N/A'

#             # Extract selling price data
#             price_element = soup1.find('div', {'class': 'price'})
#             selling_price = price_element.text.strip() if price_element else 'N/A'

#             # Extract transmission type and fuel type
#             t1 = soup1.find_all('div', {'class': 'review-overview__most-popular-content-facts-group-fact'})
#             if t1:
#                 k1 = [h1.text.strip() for h1 in t1]
#                 brand = k1[-2].split(' ')[0]
#                 fuel_type = k1[-2].split(' ')[-1]
#                 transmission_type = k1[-1]
#             else:
#                 brand = 'N/A'
#                 fuel_type = 'N/A'
#                 transmission_type = 'N/A'

#             # Extract specifications data
#             newurl1 = f"{url}/{model}/specifications"
#             r2 = requests.get(newurl1)
#             soup2 = BeautifulSoup(r2.text, 'html.parser')
#             T1 = soup2.find_all('div', {'class': 'summary-list__item'})
#             l2 = [l1.dd.text.strip() for l1 in T1]
#             vehicle_age = l2[0] if len(l2) > 0 else 'N/A'
#             km_driven = l2[1] if len(l2) > 1 else 'N/A'
#             seller_type = 'Dealer' if 'Dealer' in url else 'Individual'
#             mileage = l2[2] if len(l2) > 2 else 'N/A'
#             max_power = l2[3] if len(l2) > 3 else 'N/A'
#             seats = l2[4] if len(l2) > 4 else 'N/A'

#             # Write data to CSV
#             writer.writerow({
#                 'car_name': car_name,
#                 'brand': brand,
#                 'model': model,
#                 'vehicle_age': vehicle_age,
#                 'km_driven': km_driven,
#                 'seller_type': seller_type,
#                 'fuel_type': fuel_type,
#                 'transmission_type': transmission_type,
#                 'mileage': mileage,
#                 'engine': engine,
#                 'max_power': max_power,
#                 'seats': seats,
#                 'selling_price': selling_price
#             })


In [ ]:
df = pd.read_csv('car_recommend.csv')
df.head()

In [ ]:
df=df.drop(columns=['Unnamed: 0','seller_type'])	
df.head()

In [ ]:
# Numerical & categorical columns
numeric_features = [feature for feature in df.columns if df[feature].dtype != 'O']
categorical_features = [feature for feature in df.columns if df[feature].dtype == 'O']
print('We have {} numerical features : {}'.format(len(numeric_features), numeric_features))
print('\nWe have {} categorical features : {}'.format(len(categorical_features), categorical_features))

In [ ]:
df.info()

In [ ]:
#Missing Values 
total=df.isnull().sum().sort_values(ascending=False)
percent=(df.isnull().sum()/df.isnull().count()*100).sort_values(ascending=False)
pd.concat([total, percent],axis=1, keys=['Total', 'Percent'])

In [ ]:
df.duplicated().sum()
df=df.drop_duplicates()

In [ ]:
df.describe()

In [ ]:
# count plot (categorical)
sns.countplot(data=df,x=df['brand'])
plt.xlabel('Brand')
plt.ylabel('Count')
plt.title('Count plot of Car Brands')
plt.xticks(rotation=90)
plt.savefig(r"D:/fig3.png",bbox_inches='tight')
plt.show()

In [ ]:
# Better readability
df1=df.groupby('brand')['selling_price'].max()
 #Create the bar chart
plt.figure(figsize=(8, 4))
plt.bar(df1.index, df1.values)
plt.xlabel('brands')
plt.ylabel('Max Selling Price')
plt.title('Brands vs. their max Selling Price')
plt.xticks(rotation=90)
plt.savefig(r"D:/fig2.png",bbox_inches='tight')
plt.show()

In [ ]:
# Better readability
sorted_car_names = sorted(df['car_name'].unique()[:40])
car_prices = []
for car_name in sorted_car_names:
    car_prices.append(df[df['car_name'] == car_name]['selling_price'].values[0])
# Create the bar chart
fig, ax = plt.subplots(figsize=(8, 3))
ax.bar(sorted_car_names, car_prices)
ax.set_xlabel('Car Name')
ax.set_ylabel('Selling Price')
ax.set_title('Cars(top 40) vs. Selling Price')
ax.tick_params(axis='x', rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(x="brand", y="selling_price", data=df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
sns.relplot(kind='scatter',x="km_driven", y="selling_price", hue="fuel_type", data=df)

In [ ]:
seat_counts = df['seats'].value_counts()

wedges, texts= plt.pie(
    seat_counts,
    labels=None,   # remove labels from the pie itself
    startangle=140
)

# add legend with seat numbers
plt.legend(
    wedges, 
    seat_counts.index,
    title="Number of Seats",
    loc="best"
)

plt.title("Distribution of Seats w.r.t. Number of Cars", fontsize=14)
plt.show()


In [ ]:

# select numeric columns
numeric_df = df.select_dtypes(include=['int64', 'float64'])

# calculate correlation
corr = numeric_df.corr()

# plot heatmap
plt.figure(figsize=(10,8))
sns.heatmap(corr, annot=True, cmap="coolwarm")
plt.title("Correlation Heatmap of Numeric Features", fontsize=14)
plt.savefig(r"D:/fig1")
plt.show()



In [ ]:
categorical_columns = ['car_name', 'brand', 'model', 'fuel_type', 'transmission_type']

# Create a dictionary to store the label encoder objects
label_encoders = {}

# Loop through the categorical columns and fit label encoder to each column
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    # Store the label encoder object in the dictionary
    label_encoders[col] = le

df.head()


In [ ]:
X = data.drop('selling_price', axis=1)
y = data['selling_price']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
#machine learning algorithms
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

evs_rf = explained_variance_score(y_test, rf_pred)
r2_rf = r2_score(y_test, rf_pred)

print(f"Random Forest Explained Variance Score: {evs_rf:.3f}")
print(f"Random Forest R2 Score: {r2_rf:.3f}")


In [ ]:
xgb_model = XGBRegressor(random_state=42, n_estimators=100)
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

evs_xgb = explained_variance_score(y_test, xgb_pred)
r2_xgb = r2_score(y_test, xgb_pred)

print(f"XGBoost Explained Variance Score: {evs_xgb:.3f}")
print(f"XGBoost R2 Score: {r2_xgb:.3f}")


In [ ]:
# feature importance from Random Forest
importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10,6))
sns.barplot(x=importances[indices], y=X.columns[indices])
plt.title("Random Forest Feature Importances")
plt.savefig(r"D:/fig4.png",bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(y_test, rf_pred, alpha=0.5, label="Random Forest")
plt.scatter(y_test, xgb_pred, alpha=0.5, label="XGBoost", color="orange")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--')
plt.xlabel("Actual Selling Price")
plt.ylabel("Predicted Selling Price")
plt.title("Predicted vs Actual Selling Prices")
plt.legend()
plt.savefig(r"D:/fig5.png",bbox_inches='tight')
plt.show()

In [ ]:
summary_df = pd.DataFrame({
    'Model': ['Random Forest', 'XGBoost'],
    'Explained Variance Score': [evs_rf, evs_xgb],
    'R2 Score': [r2_rf, r2_xgb]
})

print(summary_df.to_markdown(index=False))

In [ ]:
#deep learning 
X = df.drop(['car_name'], axis=1).values
y = df['car_name'].values

In [ ]:
#ANN training
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Feature scaling
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Encoding the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# Define the model architecture
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))  # Adding dropout for regularization
model.add(Dense(64, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))  # Output layer with number of classes

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

# Train the model with early stopping
h1=model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping])


In [ ]:
# Evaluate the model ANN
scores = model.evaluate(X_test, y_test)
print("Accuracy of ANN: ")
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
# Evaluate the model
loss, mse = model.evaluate(X_test, y_test)
print("Model Loss: {:.4f}".format(loss))
print("MSE: {:.2f}".format(mse))

In [ ]:
history = h1

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy (ANN)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss (ANN)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()



In [ ]:
# RNN training
from tensorflow.keras.layers import LSTM

# Define the model architecture
model_rnn = Sequential()
model_rnn.add(LSTM(128, input_shape=(X_train.shape[1], 1), activation='relu'))  # LSTM layer with 128 units
model_rnn.add(Dense(64, activation='relu'))
model_rnn.add(Dense(len(label_encoder.classes_), activation='softmax'))  # Output layer with number of classes

# Compile the model
model_rnn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Reshape input data for LSTM
X_train_rnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))  # Reshape input to be [samples, time steps, features]
X_test_rnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Train the model with early stopping
h2=model_rnn.fit(X_train_rnn, y_train, epochs=100, batch_size=64, validation_data=(X_test_rnn, y_test), callbacks=[early_stopping])


In [ ]:
# Evaluate the model RNN
scores = model_rnn.evaluate(X_test_rnn, y_test)
print("\nAccuracy of RNN:")
print("%s: %.2f%%" % (model_rnn.metrics_names[1], scores[1]*100))
# Evaluate the model RNN
loss = model.evaluate(X_test, y_test)[0]
mse = model.evaluate(X_test, y_test)[1]
print("RNN Model Loss: {:.4f}".format(loss))
print("MSE: {:.2f}".format(mse))

In [ ]:
# Train the model with early stopping
history_rnn = h2

# Plot training & validation accuracy values
plt.plot(history_rnn.history['accuracy'])
plt.plot(history_rnn.history['val_accuracy'])
plt.title('RNN Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history_rnn.history['loss'])
plt.plot(history_rnn.history['val_loss'])
plt.title('RNN Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [ ]:
def ann(arr):
   X_new_scaled = scaler.transform(arr)
   predictions_probabilities = model.predict(X_new_scaled)
   predicted_classes1 = predictions_probabilities.argmax(axis=-1)
   print("Car prediction by ANN")
   predicted_car_names = label_encoders['car_name'].inverse_transform(predicted_classes1)
   print(predicted_car_names)
########################################################################################
def rnn(arr):
   X_new_scaled = scaler.transform(arr)
   X_new_rnn = X_new_scaled.reshape((X_new_scaled.shape[0], X_new_scaled.shape[1], 1))
   predictions = model_rnn.predict(X_new_rnn)
   predicted_classes3 = np.argmax(predictions, axis=1)
   print("\n\nCar prediction by RNN")
   predicted_car_names = label_encoders['car_name'].inverse_transform(predicted_classes3)
   print(predicted_car_names)
##########################################################################################

def cosine(arr):
    cos_similarities = cosine_similarity(arr,X)
    most_similar_index = np.argmax(cos_similarities, axis=1)
    predicted_classes2 = df.iloc[most_similar_index]['car_name'].values
    print("\n\nCar prediction by Cosine Similarity")
    predicted_car_names = label_encoders['car_name'].inverse_transform(predicted_classes2)
    print(predicted_car_names)


In [ ]:
# User Interface Defaults
default_brand = df['brand'].mode()[0]  # Most frequent brand
default_model = df['model'].mode()[0]  # Most frequent model
default_vehicle_age = int(df['vehicle_age'].mean())  # Average vehicle age
default_fuel_type = df['fuel_type'].mode()[0]  # Most frequent fuel type
default_seats = 5  # Default number of seats
default_km_driven = int(df['km_driven'].mean())  # Average km driven
default_transmission_type = 'Manual'  # Default transmission type
default_mileage = df['mileage'].mean()  # Average mileage
default_engine_size = df['engine'].mode()[0]  # Most frequent engine size
default_max_power = df['max_power'].mean()  # Average max power
default_price = int(df['selling_price'].mean())  # Average selling price

# Print unique brands
total_brands = df['brand'].unique()
brand = label_encoders['brand'].inverse_transform(total_brands)

fuel_type = df['fuel_type'].unique()
fuel_type1 = label_encoders['fuel_type'].inverse_transform(fuel_type)

transmission = df['transmission_type'].unique()
transmission1 = label_encoders['transmission_type'].inverse_transform(transmission)

# Ask for user inputs
for i, b in enumerate(brand):
    print(f"{i}: {b}")
brand_input = input("Enter the number corresponding to the brand (or press Enter for default): ").strip()
default_brand = total_brands[int(brand_input)] if brand_input else default_brand

# Ask for user input for fuel type
print("Please select the fuel type of the car (or press Enter for default):")
for i, ft in enumerate(fuel_type1):
    print(f"{i}: {ft}")
fuel_input = input("Enter the number corresponding to the fuel type: ").strip()
default_fuel_type = fuel_type[int(fuel_input)] if fuel_input else default_fuel_type

# Ask for user input for transmission type
print("Please select the transmission type of the car (or press Enter for default):")
for i, t in enumerate(transmission1):
    print(f"{i}: {t}")
transmission_input = input("Enter the number corresponding to the transmission type: ").strip()
default_transmission_type = transmission[int(transmission_input)] if transmission_input else default_transmission_type


# Ask for user input for vehicle age
vehicle_age_input = input("Enter your vehicle age requirement (or press Enter for default): ").strip()
default_vehicle_age = int(vehicle_age_input) if vehicle_age_input else default_vehicle_age

# Ask for user input for maximum km driven
km_driven_input = input("Enter maximum km driven by your vehicle (or press Enter for default): ").strip()
default_km_driven = int(km_driven_input) if km_driven_input else default_km_driven

# Ask for user input for maximum mileage capacity
mileage_input = input("Enter maximum mileage capacity for your vehicle (or press Enter for default): ").strip()
default_mileage = mileage_input if mileage_input else default_mileage

# Ask for user input for engine size
engine_size_input = input("Enter engine size for your vehicle (or press Enter for default): ").strip()
default_engine_size = engine_size_input if engine_size_input else default_engine_size

# Ask for user input for max power
max_power_input = input("Enter max power for your vehicle (or press Enter for default): ").strip()
default_max_power = max_power_input if max_power_input else default_max_power

default_price=input("enter your max price range for your vehicle : ")



user_inputs = [
    default_brand,
    default_model,
    default_vehicle_age,
    default_fuel_type,
    default_seats,
    default_km_driven,
    default_transmission_type,
    default_mileage,
    default_engine_size,
    default_max_power,
    default_price
]

# Convert all the inputs to the appropriate data types before sending them to the model
user_inputs = [
    str(user_inputs[0]),  # Brand (convert to string)
    str(user_inputs[1]),  # Model (convert to string)
    int(user_inputs[2]),  # Vehicle Age (convert to integer)
    str(user_inputs[3]),  # Fuel Type (convert to string)
    int(user_inputs[4]),  # Seats (convert to integer)
    int(user_inputs[5]),  # KM Driven (convert to integer)
    str(user_inputs[6]),  # Transmission Type (convert to string)
    float(user_inputs[7]),  # Mileage (convert to float)
    int(user_inputs[8]),  # Engine Size (convert to integer)
    float(user_inputs[9]),  # Max Power (convert to float)
    int(user_inputs[10])  # Price (convert to integer)
]
user_inputs1 = [user_inputs]
ann(user_inputs1)
rnn(user_inputs1)
cosine(user_inputs1)